## 训练循环与双重早停策略

本模块实现了 PyTorch 训练与验证的标准 Epoch 循环，并集成了一种鲁棒的**双重早停 (Early Stopping)** 机制，以优化收敛和防止过拟合。

### 核心流程

1.  **数据预处理：**
    * 训练和验证阶段均对输入张量进行了转置 (`permute`)，从 `(B, H, W, C)` 转换为 PyTorch 标准的 `(B, C, H, W)` 格式。
    * 仅提取前三个通道 (`inputs[:, :3, :, :]`)，表明输入图像可能为四通道或更高通道，但模型仅使用 RGB（或前三通道）。
2.  **优化与调度：**
    * 在**每个批次 (Batch)** 训练结束后，调用 `optimizer.step()` 更新参数，并调用 **`scheduler.step()`** 更新学习率（与 OneCycleLR 策略匹配）。
3.  **性能评估：**
    * 在每个 Epoch 结束后，计算并打印训练损失/准确率和验证损失/准确率。

### 双重早停机制

模型训练的停止条件由两个策略独立控制：

| 策略 | 触发条件 | 效果 |
| :--- | :--- | :--- |
| **策略 1: 绝对损失阈值** | 验证损失 (`val_epoch_loss`) **$\le$ 预设阈值** (`BEST_LOSS_THRESHOLD=0.01`) | **强制停止**。一旦损失达到极佳水平，立即保存模型并结束训练。 |
| **策略 2: 基于 Patience** | 连续 `patience` (40) 个 Epoch 验证损失未改善。 | **标准早停**。在等待期后停止训练，防止过拟合，并保存迄今为止**最佳**损失的模型。 |

**模型保存：** 模型权重在以下两种情况被保存为 `"best_model.pth"`：
1. 策略 1 触发（达到绝对阈值）。
2. 策略 2 中，当前验证损失打破历史最低记录 (`best_val_loss`) 时。

In [ ]:
# 使用的前置条件
patience = 40 # 连续10个epoch验证损失没有改善就停止
best_val_loss = float('inf') # 初始最佳验证损失设置为无穷大
BEST_LOSS_THRESHOLD = 0.01 # 新增的停止阈值
epochs_no_improve = 0 # 记录没有改善的epoch数量

# 训练模型
for epoch in range(num_epochs):
    # 训练阶段
    model.train()
    train_losses = []
    train_accuracies = []
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.permute(0, 3, 1, 2) # 转置为 (batch_size, channels, height, width)，调整输入通道顺序
        inputs = inputs[:, :3, :, :] # 只取前三通道
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward() # 反向传播
        optimizer.step() # 更新参数
        scheduler.step() # 调度器更新学习率
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    train_accuracy = correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(train_accuracy)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

    # 验证阶段
    model.eval()
    val_loss = 0.0
    val_losses = []
    val_accuracies = []
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.permute(0, 3, 1, 2) # 转置为 (batch_size, channels, height, width)，调整输入通道顺序
            inputs = inputs[:, :3, :, :] # 只取前三通道
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_accuracy = correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_losses.append(val_epoch_loss)
    val_accuracies.append(val_accuracy)
    
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # 策略 1: 检查是否达到预设的最低损失阈值
    if val_epoch_loss <= BEST_LOSS_THRESHOLD:
        print(f"\n✨ 验证损失 {val_epoch_loss:.4f} 达到或低于阈值 {BEST_LOSS_THRESHOLD}，停止训练。")
        torch.save(model.state_dict(), "best_model.pth")
        print("模型已保存。")
        break
        
    # 策略 2: 标准的基于 patience 的早停
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        epochs_no_improve = 0
        # 保存最佳模型
        torch.save(model.state_dict(), "best_model.pth")
        print("验证损失降低，保存最佳模型。")
    else:
        epochs_no_improve += 1
        print(f"验证损失未改善，耐心计数: {epochs_no_improve}/{patience}")

    if epochs_no_improve >= patience:
        print(f"\n连续 {patience} 个epoch验证损失没有改善，停止训练。")
        break